In [8]:
import asyncio
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.kernel import Kernel

In [9]:
# This sample allows for a streaming response verus a non-streaming response
streaming = True

# Define the agent name and instructions
AGENT_NAME = "Host"
AGENT_INSTRUCTIONS = "Answer questions about the travel."


# Define a sample plugin for the sample
class TravelPlugin:
    """A sample Travel Plugin used for the concept sample."""

    @kernel_function(description="Provides a list of countries.")
    def get_countries(self) -> Annotated[str, "Returns the countries that you can book."]:
        return """
            United States
            Australia
            France
        """

    @kernel_function(description="Provides the typical travel budget for a country")
    def get_travel_budget(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the budget for the country."]:
        if country == "United States":
            return """
                Budget Travelers: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $923 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "Australia":
            return """
                Budget Travelers: Around $75 per day. This includes staying in hostels, cooking your own meals, and using public transport.
                Mid-Range Travelers: Approximately $195 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $517 per day. This includes luxury hotels, fine dining, and private transportation.
            """
        elif country == "France":
            return """
                Budget Travelers: Around $100 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
                Mid-Range Travelers: Approximately $286 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
                Luxury Travelers: About $908 per day. This includes luxury hotels, fine dining, and private transportation.            
            """
        else:
            return "Country not found"
    
    @kernel_function(description="Provides the weather for a country")
    def get_weather(self, 
        country: Annotated[str, "The name of the country"]) -> Annotated[str, "Returns the weather for the country."]:
        if country == "United States":
            return """
                The United States has a diverse climate. In general:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
                - **Summer**: Hot and humid in the south, dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).
            """
        elif country == "Australia":
            return """
                Australia experiences varied climates:
                - **Winter**: Mild in the north, cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
                - **Summer**: Hot and dry in the interior, humid in the north. Average temperatures range from 68°F (20°C) in Hobart to 91°F (33°C) in Darwin.
            """
        elif country == "France":
            return """
                France has several climate zones:
                - **Winter**: Cold in the north, mild in the south. Average temperatures range from 35.5°F (2°C) in the northeast to 48°F (9°C) in the south.
                - **Summer**: Warm and sunny. Average temperatures range from 63°F (17°C) in the north to 77°F (25°C) along the Mediterranean coast.
            """
        else:
            return "Country not found"
    

In [10]:
# A helper method to invoke the agent with the user input
async def invoke_agent(agent: ChatCompletionAgent, input: str, chat: ChatHistory) -> None:
    """Invoke the agent with the user input."""
    chat.add_user_message(input)

    print(f"# {AuthorRole.USER}: '{input}'")

    if streaming:
        contents = []
        content_name = ""
        async for content in agent.invoke_stream(chat):
            content_name = content.name
            contents.append(content)
        message_content = "".join([content.content for content in contents])
        print(f"# {content.role} - {content_name or '*'}: '{message_content}'")
        chat.add_assistant_message(message_content)
    else:
        async for content in agent.invoke(chat):
            print(f"# {content.role} - {content.name or '*'}: '{content.content}'")
        chat.add_message(content)


In [11]:
# Create the instance of the Kernel
kernel = Kernel()

service_id = "agent"
kernel.add_service(AzureChatCompletion(service_id=service_id))

settings = kernel.get_prompt_execution_settings_from_service_id(service_id=service_id)
# Configure the function choice behavior to auto invoke kernel functions
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

kernel.add_plugin(TravelPlugin(), plugin_name="menu")

# Create the agent
agent = ChatCompletionAgent(
    service_id="agent", kernel=kernel, name=AGENT_NAME, instructions=AGENT_INSTRUCTIONS, execution_settings=settings
)

# Define the chat history
history = ChatHistory()

# Respond to user input
await invoke_agent(agent, "Hello", history)
await invoke_agent(agent, "What countries can I go to?", history)
await invoke_agent(agent, "What is the budget for United States?", history)
await invoke_agent(agent, "How about Australia?", history)
await invoke_agent(agent, "What's the weather in France?", history)
await invoke_agent(agent, "Thank you", history)


# AuthorRole.USER: 'Hello'
# AuthorRole.ASSISTANT - Host: 'Hi there! How can I assist you today?'
# AuthorRole.USER: 'What countries can I go to?'
# AuthorRole.ASSISTANT - Host: 'You can consider traveling to the following countries:
- United States
- Australia
- France

Let me know if you need more information about any of these destinations!'
# AuthorRole.USER: 'What is the budget for United States?'
# AuthorRole.ASSISTANT - Host: 'Here’s a general idea of travel budgets for the United States:

- **Budget Travelers**: Around $121 per day. This includes staying in hostels, eating at budget restaurants, and using public transportation.
- **Mid-Range Travelers**: Approximately $324 per day. This covers mid-range hotels, dining at average restaurants, and some paid attractions.
- **Luxury Travelers**: About $923 per day. This includes luxury hotels, fine dining, and private transportation.

If you need further details or help with planning, feel free to ask!'
# AuthorRole.USER: 'How abou

In [12]:
is_complete: bool = False
while not is_complete:
    user_input = input("User:> ")
    if not user_input:
        continue

    if user_input.lower() == "exit":
        is_complete = True
        break

    history.add_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

    async for response in agent.invoke(history=history):
        print(f"{response.content}")

You can consider traveling to the following countries:
- United States
- Australia
- France

Let me know if you need more information about any of these destinations!
Hi there! You can consider traveling to the following countries:
- United States
- Australia
- France

Let me know if you need more information about any of these destinations or have any other questions!
The United States has a diverse climate:

- **Winter**: It's cold in the north and mild in the south. Average temperatures range from 26.6°F (-3°C) in Alaska to 70.7°F (21.5°C) in Florida.
- **Summer**: It can be hot and humid in the south and dry in the west. Average temperatures range from 52.7°F (11.5°C) to 70.7°F (21.5°C).

If you need specific weather conditions for different regions or cities in the United States, please let me know!
Australia experiences diverse climates:

- **Winter**: It's mild in the north and cooler in the south. Average temperatures range from 46°F (8°C) in Canberra to 77°F (25°C) in Darwin.
